# Goal

The goal of this notebook is to query the AWS RDS of Amazon Video Downloads.

# Environment Setup and Dependencies

In [ ]:
# Import AWS RDS configuration
from google.colab import files
src = list(files.upload().values())[0]
open('config.py','wb').write(src)
from config import username, password, rds_url

Saving config.py to config (3).py


In [ ]:
# Dependencies
import os

# set spark version
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark


# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (110 kB/s)
Reading package lis

In [ ]:
# postgres connection
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-03 15:56:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-08-03 15:56:53 (10.7 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [ ]:
# setup pyspark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonQuery")\
  .config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar")\
  .getOrCreate()

# Query Postgres Database

In [ ]:
# Configure settings for RDS
# use imported variables from config.py
jdbc_url=f"jdbc:postgresql://{rds_url}"
config = {"user": username, 
          "password": password, 
          "driver":"org.postgresql.Driver"}

# Direct SQL Query

In [ ]:
query = "(select * from vine_table) as Vine"

In [ ]:
df2 = spark.read.jdbc(url=jdbc_url, table=query, properties=config)

In [ ]:
df2.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006IKFAOWPPC|          5|            0|          0|   N|
|R1006V1JWI9VYM|          4|            0|          0|   N|
|R100DP5939VLI1|          4|            0|          0|   N|
|R100SNQ08T1G0W|          3|            1|          1|   N|
|R100XEX1Y95QEO|          5|            0|          0|   N|
|R100ZVN2C953CB|          4|            0|          0|   N|
|R1011QRZ1IRBX3|          5|            3|          3|   N|
|R101EJU3ET0EZ8|          4|            0|          0|   N|
|R101KJVKVYEVGA|          4|            0|          0|   N|
|R101O01CUFADI0|          5|            0|          2|   N|
|R101OFEXKHXSGX|          5|            0|          0|   N|
|R101TKMOB0YTJ5|          4|            0|          0|   N|
|R101W2SHXLSF95|          5|            0|          0|   N|
|R1022V48MM5TGY|          5|            

# Table-only Queary

In [ ]:
# Write DataFrame to vine_table in RDS

df = spark.read.format("jdbc")\
    .option("url", jdbc_url)\
    .option("driver", "org.postgresql.Driver")\
    .option("dbtable", "vine_table")\
    .option("user", username)\
    .option("password", password)\
    .load()

In [ ]:
df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006IKFAOWPPC|          5|            0|          0|   N|
|R1006V1JWI9VYM|          4|            0|          0|   N|
|R100DP5939VLI1|          4|            0|          0|   N|
|R100SNQ08T1G0W|          3|            1|          1|   N|
|R100XEX1Y95QEO|          5|            0|          0|   N|
|R100ZVN2C953CB|          4|            0|          0|   N|
|R1011QRZ1IRBX3|          5|            3|          3|   N|
|R101EJU3ET0EZ8|          4|            0|          0|   N|
|R101KJVKVYEVGA|          4|            0|          0|   N|
|R101O01CUFADI0|          5|            0|          2|   N|
|R101OFEXKHXSGX|          5|            0|          0|   N|
|R101TKMOB0YTJ5|          4|            0|          0|   N|
|R101W2SHXLSF95|          5|            0|          0|   N|
|R1022V48MM5TGY|          5|            

# Comments

*  Additional filters can be added and won't be executed until .show(), count(), collect(), take(n), top(), countByValue(), reduce(), fold(), aggregate(), foreach()

_https://data-flair.training/blogs/spark-rdd-operations-transformations-actions/_


# Recreate Original Dataframe